In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 
from tensorflow import keras

In [2]:
import cv2
from keras.callbacks import EarlyStopping
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Rescaling, BatchNormalization, Concatenate
from keras.models import Model

In [3]:
df=pd.read_csv('socal2.csv')
df.head()

,image_id,street,citi,n_citi,bed,bath,sqft,price
0,0,1317 Van Buren Avenue,"Salton City, CA",317,3,2.0,1560,201900
1,1,124 C Street W,"Brawley, CA",48,3,2.0,713,228500
2,2,2304 Clark Road,"Imperial, CA",152,3,1.0,800,273950
3,3,755 Brawley Avenue,"Brawley, CA",48,3,1.0,1082,350000
4,4,2207 R Carrillo Court,"Calexico, CA",55,4,3.0,2547,385100


In [4]:
from ydata_profiling import ProfileReport
report = ProfileReport(df)
report

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
df['street'].nunique()

12401

In [6]:
df['street']=df['street'].str.split(' ',n=1).str[1]

In [7]:
df['street'].nunique()

10467

In [8]:
df['street'].value_counts()

street
not provided          23
Avenida Majorca       17
Calle Aragon          16
Avenida Castilla      12
Woodland Drive        12
                      ..
Lamont Drive           1
Stacey Avenue          1
Golden Eagle Court     1
Calle Sinaloa          1
Eastbourne Bay         1
Name: count, Length: 10467, dtype: int64

In [9]:
df = df.replace({'street': df['street'].value_counts().to_dict()})

In [10]:
df['citi'].nunique()

415

In [11]:
df

,image_id,street,citi,n_citi,bed,bath,sqft,price
0,0,1,"Salton City, CA",317,3,2.0,1560,201900
1,1,1,"Brawley, CA",48,3,2.0,713,228500
2,2,1,"Imperial, CA",152,3,1.0,800,273950
3,3,2,"Brawley, CA",48,3,1.0,1082,350000
4,4,1,"Calexico, CA",55,4,3.0,2547,385100
...,...,...,...,...,...,...,...,...
15469,15469,1,"Moorpark, CA",227,5,4.1,4092,949000
15470,15470,2,"Thousand Oaks, CA",372,5,3.0,2773,949900
15471,15471,1,"Santa Paula, CA",338,3,2.0,1576,920000
15472,15472,1,"Oak View, CA",253,4,2.0,2086,997000


In [12]:
df = pd.get_dummies(df, columns=['citi'])
df = df.astype(int)

In [13]:
df=df.drop('n_citi',axis=1)
df


,image_id,street,bed,bath,sqft,price,"citi_29 Palms, CA","citi_Acton, CA","citi_Adelanto, CA","citi_Agoura Hills, CA",...,"citi_Wildomar, CA","citi_Wilmington, CA","citi_Winchester, CA","citi_Winnetka, CA","citi_Wofford Heights, CA","citi_Woodland Hills, CA","citi_Wrightwood, CA","citi_Yorba Linda, CA","citi_Yucaipa, CA","citi_Yucca Valley, CA"
0,0,1,3,2,1560,201900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,3,2,713,228500,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,1,3,1,800,273950,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,2,3,1,1082,350000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,1,4,3,2547,385100,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15469,15469,1,5,4,4092,949000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15470,15470,2,5,3,2773,949900,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15471,15471,1,3,2,1576,920000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
15472,15472,1,4,2,2086,997000,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
X=df.drop(columns=['price']).to_numpy()
y=df['price'].to_numpy()

In [17]:
X

array([[0.0000e+00, 1.0000e+00, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.0000e+00, 1.0000e+00, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [2.0000e+00, 1.0000e+00, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       ...,
       [1.5471e+04, 1.0000e+00, 3.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.5472e+04, 1.0000e+00, 4.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00],
       [1.5473e+04, 1.0000e+00, 2.0000e+00, ..., 0.0000e+00, 0.0000e+00,
        0.0000e+00]])

In [18]:
img_folder= 'C:\\Users\\peyush.jindal\\Desktop\\New folder\\socal2\\socal_pics'
img_height = 128
img_width = 128

import os
files = os.listdir(img_folder)
files.sort(key=lambda x: int(x.split('.')[0]))

img_list = []

for file in files:
    img = cv2.imread(os.path.join(img_folder, file))
    img = cv2.resize(img,(img_height, img_width))
    img_list.append(img)
    
img_features = np.asarray(img_list)

In [19]:
rng = np.random.default_rng(seed=13)

size = len(X)

p = rng.permutation(size)

X = X[p]
z = img_features[p]
y = y[p]

In [20]:
val_idx = int(size * 0.6)
test_idx = int(size * 0.8)
X_train,X_val,X_test = np.split(X,[val_idx, test_idx])
z_train,z_val,z_test = np.split(z,[val_idx, test_idx])
y_train,y_val,y_test = np.split(y,[val_idx, test_idx])

In [21]:
from keras import Sequential
model_X= Sequential([
    Dense(1024,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(1024,activation='relu')
])
model_X.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1024)              431104    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
Total params: 1,480,704
Trainable params: 1,480,704
Non-trainable params: 0
_________________________________________________________________


In [22]:
model_z= Sequential([
    Rescaling(1./255,input_shape=(img_height, img_width, 3)),
    Conv2D(64, 3, padding='same', activation='relu'),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(padding='same'),
    BatchNormalization(),
    Conv2D(128, 3, padding='same', activation='relu'),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(padding='same'),
    BatchNormalization(),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    Conv2D(256, 3, padding='same', activation='relu'),
    MaxPooling2D(padding='same'),
    BatchNormalization(),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    Conv2D(512, 3, padding='same', activation='relu'),
    MaxPooling2D(padding='same'),
    BatchNormalization(),
    Conv2D(1024, 3, padding='same', activation='relu'),
    Conv2D(1024, 3, padding='same', activation='relu'),
    Conv2D(1024, 3, padding='same', activation='relu'),
    MaxPooling2D(padding='same'),
    BatchNormalization(),
    Dropout(0.25),
    Flatten(),
    Dense(1024, activation='relu'),
    Dense(1024, activation='relu'),
])

model_z.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 128, 128, 3)       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 64, 64, 64)       256       
 ormalization)                                                   
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)      

In [23]:
input = Concatenate()([model_X.output, model_z.output])
x = Dense(4096, activation='relu')(input)
x = Dropout(0.1)(x)
x = Dense(4096, activation='relu')(x)
x = Dense(1, activation='linear')(x)

model = Model([model_X.input, model_z.input], x)

model.compile(optimizer='adam', loss='mse',metrics=['mae', 'mape'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rescaling_input (InputLayer)   [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 128, 128, 3)  0           ['rescaling_input[0][0]']        
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 64  1792        ['rescaling[0][0]']              
                                )                                                                 
                                                                                              

In [24]:
epochs = 100
batch_size = 64

early_stopping = EarlyStopping(patience=20, verbose=2, restore_best_weights=True)
    
history = model.fit(
    [X_train, z_train], 
    y_train, 
    validation_data=([X_val, z_val], y_val),
    epochs=epochs,
    batch_size=batch_size,
    callbacks=early_stopping
)

Epoch 1/100
146/146 [==============================] - 1638s 11s/step - loss: 490287.8125 - mae: 65.8000 - mape: 5198325.0000 - val_loss: 7.5541 - val_mae: 2.3328 - val_mape: 2246678.5000
Epoch 2/100
146/146 [==============================] - 1622s 11s/step - loss: 11.3866 - mae: 2.3192 - mape: 1533074.0000 - val_loss: 0.5241 - val_mae: 0.6379 - val_mape: 1414897.7500
Epoch 3/100
146/146 [==============================] - 1644s 11s/step - loss: 4.8892 - mae: 1.5326 - mape: 804286.8750 - val_loss: 1.2678 - val_mae: 0.9535 - val_mape: 750381.3125
Epoch 4/100
146/146 [==============================] - 1649s 11s/step - loss: 1.1282 - mae: 0.7410 - mape: 474571.1250 - val_loss: 0.4823 - val_mae: 0.5703 - val_mape: 363939.1875
Epoch 5/100
146/146 [==============================] - 1647s 11s/step - loss: 0.5635 - mae: 0.5238 - mape: 467138.1875 - val_loss: 0.3437 - val_mae: 0.5106 - val_mape: 521658.0312
Epoch 6/100
146/146 [==============================] - 1643s 11s/step - loss: 0.2848 - ma

In [25]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model mean squared error')
plt.ylabel('mse')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

C:\Users\peyush.jindal\AppData\Local\Temp\ipykernel_25856\2335868749.py:7: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [26]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('model mean absolute error')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

C:\Users\peyush.jindal\AppData\Local\Temp\ipykernel_25856\4118157523.py:7: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [27]:
plt.plot(history.history['mape'])
plt.plot(history.history['val_mape'])
plt.title('model mean absolute percentage error')
plt.ylabel('mape')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

C:\Users\peyush.jindal\AppData\Local\Temp\ipykernel_25856\3302120241.py:7: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [28]:
y_pred_nn = model.predict([X_test, z_test])

97/97 [==============================] - 86s 875ms/step


In [29]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_absolute_percentage_error,mean_squared_error
metrics_nn = model.evaluate([X_test, z_test], y_test)
metrics_nn.append(r2_score(y_test, y_pred_nn))

97/97 [==============================] - 84s 867ms/step - loss: 0.0344 - mae: 0.1336 - mape: 290982.8125


In [30]:
def scatterplot_true_and_pred(y_true, y_pred, title) -> None:

    ax = sns.scatterplot(x=y_true.reshape(-1), y=y_pred.reshape(-1))
    ax.set_title(title)
    ax.set(xlabel='True Value', ylabel='Predicted Value')

    max_elem = max([y_true.max(), y_pred.max()])
    diag_values = np.linspace(0, max_elem, 1000)
    plt.plot(diag_values, diag_values, color='magenta', linestyle='--')
    plt.show()

In [31]:

scatterplot_true_and_pred(y_true=y_test, y_pred=y_pred_nn, title='Performance of NN')

C:\Users\peyush.jindal\AppData\Local\Temp\ipykernel_25856\1575728957.py:10: UserWarning: Matplotlib is currently using module://matplotlib_inline.backend_inline, which is a non-GUI backend, so cannot show the figure.
  plt.show()


In [33]:
from xgb import XGBRegressor
xgbr = XGBRegressor(n_estimators=500,
                    early_stopping_rounds=10)

ModuleNotFoundError: No module named 'xgb'

In [ ]:
xgbr.fit(X_train, 
             y_train, 
             eval_set=[(X_val, y_val)])

In [ ]:
y_pred_xgb = xgbr.predict(X_test)

In [ ]:
metrics_xgb = []
metrics_xgb.append(mean_squared_error(y_test, y_pred_xgb))
metrics_xgb.append(mean_absolute_error(y_test, y_pred_xgb))
metrics_xgb.append(mean_absolute_percentage_error(y_test, y_pred_xgb))
metrics_xgb.append(r2_score(y_test, y_pred_xgb))

In [ ]:
scatterplot_true_and_pred(y_true=y_test, y_pred=y_pred_xgb, title='Performance of XGBoost')

In [ ]:
print('\tNN\t\t\tXGBoost')
print(f'MSE:\t{metrics_nn[0]}\t\t{metrics_xgb[0]}')
print(f'MAE:\t{metrics_nn[1]}\t\t{metrics_xgb[1]}')
print(f'MAPE:\t{metrics_nn[2]}\t{metrics_xgb[2]}')
print(f'R2:\t{metrics_nn[3]}\t{metrics_xgb[3]}')

In [34]:
print('\tNN\t\t\tXGBoost')
print(f'MSE:\t{metrics_nn[0]}')
print(f'MAE:\t{metrics_nn[1]}')
print(f'MAPE:\t{metrics_nn[2]}')
print(f'R2:\t{metrics_nn[3]}')

	NN			XGBoost
MSE:	0.034373119473457336
MAE:	0.13360579311847687
MAPE:	290982.8125
R2:	0.1728722017331662
